# 준비

In [16]:
!pip install pulp

In [17]:
import pandas as pd
from google.colab import drive
import numpy as np

import itertools
from scipy.spatial import distance_matrix
import matplotlib
import matplotlib.pylab as plt
import seaborn as sns
import pulp

import warnings
warnings.filterwarnings("ignore")

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
center = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/센터.csv")
od_mat = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/OD_Matrix.txt")
stop_data = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/착지.csv")
vehicles = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/vehicles.csv")

order_Monday = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/주문_월요일.csv")
order_Tuesday = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/주문_화요일.csv")
order_Wednesday = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/주문_수요일.csv")
order_Thursday = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/주문_목요일.csv")
order_Friday = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/주문_금요일.csv")
order_Saturday = pd.read_csv("/content/drive/MyDrive/Colab_project/CJ대한통운/주문_토요일.csv")

preprocessing

In [19]:
def orders_preprocessing(order_df):
  #drop terminal demand
  df = order_df.copy()
  df.dropna(inplace=True)

  #stop_id: str to int
  df['Stop_ID'].values.tolist()
  tmp=[]
  for i in df['Stop_ID'].values.tolist():
    tmp.append(int(i[2:]))
  df['Stop_ID'] = tmp

  #merge duplicated order
  df_sorted = df.sort_values('Stop_ID')
  df_adj = pd.DataFrame(columns = df.columns)
  df_adj.loc[0] = df_sorted.loc[0]

  for i in range(1,df['Stop_ID'].nunique()):
    tmp = df_sorted.loc[i]
    if df_adj.loc[len(df_adj)-1]['Stop_ID'] != tmp['Stop_ID']:
      df_adj.loc[i] = tmp
    else:
      df_adj.loc[len(df_adj)-1]['weight'] += tmp['weight']
      df_adj.loc[len(df_adj)-1]['plt'] += tmp['plt']

  df_adj.head()
  return df_adj

In [20]:
order_Monday_adj = orders_preprocessing(order_Monday)
order_Tuesday_adj = orders_preprocessing(order_Tuesday)
order_Wednesday_adj = orders_preprocessing(order_Wednesday)
order_Thursday_adj = orders_preprocessing(order_Thursday)
order_Friday_adj = orders_preprocessing(order_Friday)
order_Saturday_adj = orders_preprocessing(order_Saturday)

od_mat

In [21]:
od_mat_new=od_mat.copy()

#Terminal node
od_mat_new.loc[od_mat['OriginID']=='Terminal', 'OriginID'] = 'S_0'
od_mat_new.loc[od_mat['DestinationID']=='Terminal', 'DestinationID'] = 'S_0'

#node: str to int
od_mat_new['OriginID'].values.tolist()
tmp=[]
for i in od_mat_new['OriginID'].values.tolist():
  tmp.append(int(i[2:]))
od_mat_new['OriginID'] = tmp

tmp=[]
for i in od_mat_new['DestinationID'].values.tolist():
  tmp.append(int(i[2:]))
od_mat_new['DestinationID'] = tmp

#sort by values
od_mat_new.sort_values(by=['OriginID','DestinationID'], inplace=True)
od_mat_new.reset_index(drop=True, inplace=True)

distance matrix

In [22]:
od_len=len(od_mat_new)
def get_dist_mat(stop_list):
  stop_list.sort()
  n_node = len(stop_list)
  d_mat = np.zeros([n_node, n_node])
  t_mat = np.zeros([n_node, n_node])

  for x,i in enumerate(stop_list):
    if x == len(stop_list)-1:
      break

    itr = od_mat_new[od_mat_new['OriginID']==i].index[0]
    for y,j in enumerate(stop_list):
      if y<=x:
        continue

      while od_mat_new.loc[itr]['DestinationID'] != j:
        itr+=1
        if itr>od_len:
          print(f'destination: {j, itr} wrong!')
          break

      d_mat[x][y] = od_mat_new.loc[itr]['Total_Distance']
      d_mat[y][x] = od_mat_new.loc[itr]['Total_Distance']
      t_mat[x][y] = od_mat_new.loc[itr]['Total_Time']
      t_mat[y][x] = od_mat_new.loc[itr]['Total_Time']

  return d_mat,t_mat

# K-Means Clustering

distance*weight

In [23]:
!pip install haversine

In [24]:
from haversine import haversine

order_list = [order_Monday, order_Tuesday, order_Wednesday, order_Thursday, order_Friday, order_Saturday]

def calculate_means(df):
    wmean = df['weight'].mean()
    pltmean = df['plt'].mean()
    return wmean, pltmean


def calc_weighted_d(i, j, order_day, wmean, pltmean):
    a = (order_day.loc[i]['Y'], order_day.loc[i]['X'])
    b = (order_day.loc[j]['Y'], order_day.loc[j]['X'])

    wa = order_day.loc[i]['weight'] / wmean * order_day.loc[i]['plt'] / wmean
    wb = order_day.loc[j]['weight'] / wmean * order_day.loc[j]['plt'] / pltmean
    w = wa * wb / (wa + wb)  # 조화평균

    return haversine(a, b) / w

for order_day in order_list:
    wmean, pltmean = calculate_means(order_day)
    # 여기서 i와 j는 데이터프레임의 인덱스라고 가정
    i, j = 0, 1  # 실제로는 필요한 인덱스를 사용
    result = calc_weighted_d(i, j, order_day, wmean, pltmean)



distance base

In [25]:
from sklearn.preprocessing import StandardScaler

order_adj_list = [order_Monday_adj, order_Tuesday_adj, order_Wednesday_adj, order_Thursday_adj, order_Friday_adj, order_Saturday_adj]

def cord_scaled_day(order_adj):
  cord = order_adj.dropna()[['X','Y']].values

  scaler = StandardScaler()
  cord_scaled = scaler.fit_transform(cord)

  return cord_scaled

In [26]:
from sklearn.cluster import KMeans

kmc_k=30
kmc_Monday = KMeans(n_clusters=kmc_k, random_state=12345).fit(cord_scaled_day(order_Monday_adj))
tmp_Monday = list(kmc_Monday.labels_)

order_Monday_cluster = order_Monday_adj.copy()
order_Monday_cluster['cluster_id'] = tmp_Monday
order_Monday_cluster.sort_values('cluster_id', inplace=True)
order_Monday_cluster.reset_index(drop=True, inplace=True)
order_Monday_cluster_groupbyid = [group for _, group in order_Monday_cluster.groupby('cluster_id')] # Group by cluster_id


kmc_Tuesday = KMeans(n_clusters=kmc_k, random_state=12345).fit(cord_scaled_day(order_Tuesday_adj))
tmp_Tuesday = list(kmc_Tuesday.labels_)

order_Tuesday_cluster = order_Tuesday_adj.copy()
order_Tuesday_cluster['cluster_id'] = tmp_Tuesday
order_Tuesday_cluster.sort_values('cluster_id', inplace=True)
order_Tuesday_cluster.reset_index(drop=True, inplace=True)
order_Tuesday_cluster_groupbyid = [group for _, group in order_Tuesday_cluster.groupby('cluster_id')]


kmc_Wednesday = KMeans(n_clusters=kmc_k, random_state=12345).fit(cord_scaled_day(order_Wednesday_adj))
tmp_Wednesday = list(kmc_Wednesday.labels_)

order_Wednesday_cluster = order_Wednesday_adj.copy()
order_Wednesday_cluster['cluster_id'] = tmp_Wednesday
order_Wednesday_cluster.sort_values('cluster_id', inplace=True)
order_Wednesday_cluster.reset_index(drop=True, inplace=True)
order_Wednesday_cluster_groupbyid = [group for _, group in order_Wednesday_cluster.groupby('cluster_id')]


kmc_Thursday = KMeans(n_clusters=kmc_k, random_state=12345).fit(cord_scaled_day(order_Thursday_adj))
tmp_Thursday = list(kmc_Thursday.labels_)

order_Thursday_cluster = order_Thursday_adj.copy()
order_Thursday_cluster['cluster_id'] = tmp_Thursday
order_Thursday_cluster.sort_values('cluster_id', inplace=True)
order_Thursday_cluster.reset_index(drop=True, inplace=True)
order_Thursday_cluster_groupbyid = [group for _, group in order_Thursday_cluster.groupby('cluster_id')]


kmc_Friday = KMeans(n_clusters=kmc_k, random_state=12345).fit(cord_scaled_day(order_Friday_adj))
tmp_Friday = list(kmc_Friday.labels_)

order_Friday_cluster = order_Friday_adj.copy()
order_Friday_cluster['cluster_id'] = tmp_Friday
order_Friday_cluster.sort_values('cluster_id', inplace=True)
order_Friday_cluster.reset_index(drop=True, inplace=True)
order_Friday_cluster_groupbyid = [group for _, group in order_Friday_cluster.groupby('cluster_id')]


kmc_Saturday = KMeans(n_clusters=kmc_k, random_state=12345).fit(cord_scaled_day(order_Saturday_adj))
tmp_Saturday = list(kmc_Saturday.labels_)

order_Saturday_cluster = order_Saturday_adj.copy()
order_Saturday_cluster['cluster_id'] = tmp_Saturday
order_Saturday_cluster.sort_values('cluster_id', inplace=True)
order_Saturday_cluster.reset_index(drop=True, inplace=True)
order_Saturday_cluster_groupbyid = [group for _, group in order_Saturday_cluster.groupby('cluster_id')]


In [27]:
order_Monday_cluster['cluster_id'].value_counts(sort=False)

cluster_id
0     55
1     38
2     17
3     12
4     51
5     13
6     29
7     32
8     13
9     14
10    63
11     6
12     3
13    31
14    33
15    47
16     7
17     6
18     8
19     8
20     8
21    21
22    47
23    28
24     7
25     7
26    22
27     3
28    19
29    26
Name: count, dtype: int64

In [28]:
c_value_counts_Monday = list(order_Monday_cluster['cluster_id'].value_counts(sort=False))
c_index_Monday = [0]
for i in range(1, len(c_value_counts_Monday) + 1):
    c_index_Monday.append(sum(c_value_counts_Monday[:i]))

c_value_counts_Tuesday = list(order_Tuesday_cluster['cluster_id'].value_counts(sort=False))
c_index_Tuesday = [0]
for i in range(1, len(c_value_counts_Tuesday) + 1):
    c_index_Tuesday.append(sum(c_value_counts_Tuesday[:i]))

c_value_counts_Wednesday = list(order_Wednesday_cluster['cluster_id'].value_counts(sort=False))
c_index_Wednesday = [0]
for i in range(1, len(c_value_counts_Wednesday) + 1):
    c_index_Wednesday.append(sum(c_value_counts_Wednesday[:i]))

c_value_counts_Thursday = list(order_Thursday_cluster['cluster_id'].value_counts(sort=False))
c_index_Thursday = [0]
for i in range(1, len(c_value_counts_Thursday) + 1):
    c_index_Thursday.append(sum(c_value_counts_Thursday[:i]))

c_value_counts_Friday = list(order_Friday_cluster['cluster_id'].value_counts(sort=False))
c_index_Friday = [0]
for i in range(1, len(c_value_counts_Friday) + 1):
    c_index_Friday.append(sum(c_value_counts_Friday[:i]))

c_value_counts_Saturday = list(order_Saturday_cluster['cluster_id'].value_counts(sort=False))
c_index_Saturday = [0]
for i in range(1, len(c_value_counts_Saturday) + 1):
    c_index_Saturday.append(sum(c_value_counts_Saturday[:i]))


# GA

In [29]:
#list 안의 list 로 정렬
def sort_list(a,row_num):
  a = np.array(a).T
  a = sorted(a, key=lambda x:x[row_num])
  a = np.array(a).T
  return a

In [30]:
import itertools
import random as rd
from typing import Union
import copy

class GeneticAlgorithm:
  def __init__(self, sub_order_df, d_mat, t_mat, *args, **kwargs):
    self.sub_order_df = sub_order_df
    self.stop_list = stop_list
    self.d_mat = d_mat
    self.t_mat = t_mat
    self.n_feat = len(d_mat)-1 # terminal 제외!!
    self.population_size = kwargs['population_size']
    self.n_gen = kwargs['n_gen']
    self.init_rate = kwargs['init_rate']
    self.mutation_rate = kwargs['mutation_rate']
    self.crossover_rate = kwargs['crossover_rate']
    self.max_car_num = kwargs['max_car_num']

  def get_route_dictionary(self, chromosome):
    chromosome = sort_list(chromosome, 1)
    c_len=len(chromosome[0])
    d={};i=0
    while i<c_len:
      j=i
      while j<c_len and chromosome[1][j]==chromosome[1][i]:
        j+=1
      priority = chromosome[0][i:j]
      nodes = chromosome[2][i:j]
      tmp = sort_list([priority,nodes],0)
      d[chromosome[1][i]] = list(tmp[1])
      i=j
    return d

  def feasibility_check(self, chromosome):
    i=0; good=True;
    route_dict = ga.get_route_dictionary(chromosome)
    for v,route in route_dict.items():
      route = [0]+route+[0]
      v=int(v/100)
      weight_capa = vehicles.loc[v]['MaxCapaWeight']
      plt_capa = vehicles.loc[v]['MaxCapaPLT']

      #access restriction:
      for stop in route:
        acc_res = stop_data[stop_data['ID']=='S_'+str(stop)]['Access_restriction'].values
        if acc_res > 0:
          if int(v/100) > acc_res:
            return False

      #weight constraints:
      total_weight = 0
      for i in route:
        if i==0:
          continue
        total_weight += self.sub_order_df.loc[i-1]['weight']
      if total_weight > weight_capa:
        return False

      #plt constraints:
      total_plt = 0
      for i in route:
        if i==0:
          continue
        total_plt += self.sub_order_df.loc[i-1]['plt']
      if total_plt > plt_capa:
        return False

      #time constraints:
      current_time=500;conv=100/60

      for i in range(len(route)-1):
        moving_time = t_mat[route[i]][route[i+1]] #이동시간
        unloading_time=0
        if route[i+1]!=0:
          unloading_time = self.sub_order_df.loc[route[i+1]-1]['plt']*10*conv #하역시간
        current_time += moving_time #ETA
        #stop constraints:
        node_start = stop_data[stop_data['ID']=='S_'+str(route[i+1])]['Start_time']
        node_end = stop_data[stop_data['ID']=='S_'+str(route[i+1])]['End_time']
        if current_time < node_start.values:
          current_time = float(node_start.values)
        if current_time > node_end.values:
          good = False
          break
        current_time+=unloading_time

      if current_time > 2200:
        good = False

      if not good:
        break
    return good

  def initialization_of_population(self, size:int)->list:
    #print('initialize the population')
    population = []
    while len(population) < size:
      p_chromosome = [i+1 for i in range(self.n_feat)]
      rd.shuffle(p_chromosome)

      c_chromosome = []
      for _ in range(self.n_feat):
        c = rd.randrange(5)*100+rd.randrange(self.max_car_num) #군대식 차번호 사용!
        c_chromosome.append(c)

      index = [i+1 for i in range(len(p_chromosome))]
      if self.feasibility_check([p_chromosome, c_chromosome, index]):
        population.append([p_chromosome, c_chromosome, index])

    return population

  def fitness_score(self, population):
    scores = []
    for chromosome in population:
      d = self.get_route_dictionary(chromosome)
      total_cost=0

      #route cost
      for v,route in d.items():
        route = [0]+route+[0]
        route_cost=0
        for i in range(len(route)-1):
          route_cost += d_mat[route[i]][route[i+1]]*vehicles.loc[int(v/100)]['VariableCost'] # route cost
        total_cost += route_cost

      #fixed cost
      v_num=np.zeros([5])
      for v,route in d.items():
        v_num[int(v/100)]+=1
      fixed_cost=0; reuse_cost=0
      for i in range(5):
        fixed_cost+=vehicles.loc[i]['FixedCost']*int(v_num[i]/2)
        reuse_cost+=vehicles.loc[i]['RoundCost']*(v_num[i]%2)
      total_cost+=fixed_cost+reuse_cost

      scores.append(1000000/total_cost)

    return scores

  def selection(self, population, scores): #roulette wheel method
    population_nextgen = []; prob_list=[]; s=0
    population_nextgen.append(population[scores.index(max(scores))]) # reserve the best chromosome - elite

    for i in range(self.population_size):
      s += (scores[i]/sum(scores))
      prob_list.append(s)
    for i in range(self.population_size-1):
      p = rd.randrange(0,100)/100;j=0
      while j<self.population_size and p>prob_list[j]:
        j+=1
      population_nextgen.append(population[j])

    return population_nextgen


  def crossover(self, pop_after_sel:list,crossover_rate:float):
    pop_nextgen = copy.deepcopy(pop_after_sel)
    pop_nextgen.append(pop_nextgen[0]) # elite 보존 정책
    sex=[]
    for i in range(1,self.population_size+1):
      p = rd.randrange(0,100)/100
      if p<crossover_rate:
        sex.append(i)
    rd.shuffle(sex) #select parents who will have sex

    for i in range(0, len(sex),2):
      if i+1>=len(sex):
        break
      boy = copy.deepcopy(pop_nextgen[sex[i]])
      girl = copy.deepcopy(pop_nextgen[sex[i+1]])

      c_attempt=0
      while True: #sex until we get good kid
        #priority
        p=q=0
        while p==q:
          p = rd.randrange(0,self.n_feat)
          q = rd.randrange(0,self.n_feat)
        start=min(p,q);end=max(p,q)
        b_to_go = boy[0][start:end]
        g_to_go = girl[0][start:end]
        for j in range(len(b_to_go)):
          boy[0].remove(g_to_go[j])
          girl[0].remove(b_to_go[j])
        for j in range(len(b_to_go)):
          boy[0].insert(start+j,g_to_go[j])
          girl[0].insert(start+j,b_to_go[j])

        #car
        p=q=0
        while p==q:
          p = rd.randrange(0,self.n_feat)
          q = rd.randrange(0,self.n_feat)
        start=min(p,q);end=max(p,q)
        tmp = boy[1][start:end]
        boy[1]=boy[1][:start]+girl[1][start:end]+boy[1][end:]
        girl[1]=girl[1][:start]+tmp+girl[1][end:]

        #check feasibility:
        if not self.feasibility_check(boy) or not self.feasibility_check(girl):
          c_attempt+=1
          if c_attempt>50:
            print('no feasible solution found while performing crossover')
            break
          continue

        #test finished
        break

      #insert kids
      pop_nextgen[sex[i]] = boy
      pop_nextgen[sex[i+1]] = girl

    pop_nextgen.pop()
    return pop_nextgen


  def mutation(self, pop_after_cross:list, mutation_rate:float, n_feat:int):
    p = rd.randrange(100)/100
    if p>mutation_rate:
      return pop_after_cross
    pop_nextgen = copy.deepcopy(pop_after_cross)
    i = rd.randrange(1,self.population_size)
    #priority
    p=q=0
    while p==q:
      p = rd.randrange(self.n_feat)
      q = rd.randrange(self.n_feat)
    tmp = pop_nextgen[i][0][p]
    pop_nextgen[i][0][p] = pop_nextgen[i][0][q]
    pop_nextgen[i][0][q] = tmp

    #car
    p = rd.randrange(self.n_feat)
    pop_nextgen[i][1][p] = 100*rd.randrange(5)+rd.randrange(self.max_car_num)

    return pop_nextgen

  def find_solution(self):
    best_chromosome = []
    best_score = []
    population = self.initialization_of_population(self.population_size)
    for i in range(self.n_gen):
      scores = self.fitness_score(population)
      best_score.append(max(scores))
      best_chromosome.append(population[scores.index(max(scores))])
      print('Best score in generation',i+1,':',max(scores))
      #print(population)
      pop_after_sel = self.selection(population, scores)
      pop_after_cross = self.crossover(pop_after_sel,self.crossover_rate)
      population = self.mutation(pop_after_cross, self.mutation_rate, self.n_feat)

    return best_chromosome, best_score

In [31]:
'''
#전체 탐색
def solution(c_index, order_cluster):
  final = []
  stop_list = []
  for k,x in enumerate(c_index[1:]):
    index = [i for i in range(c_index[k],c_index[k+1])]
    print('/'*30)
    print(f'processing {k+1}th cluster. complexity: {len(index)}')

    sub_order = order_cluster.loc[index]
    sub_order.sort_values('Stop_ID', inplace=True)
    sub_order.reset_index(drop=True, inplace=True)

    stop_list = [0]+list(sub_order['Stop_ID'])
    d_mat,t_mat = get_dist_mat(stop_list)

    #maximum vehicles
    total_weight = sub_order['weight'].sum()
    total_plt = sub_order['plt'].sum()
    max_car_w = total_weight/1000
    max_car_p = total_plt/2.8
    max_car = int(max(max_car_w, max_car_p)+1)
    if max_car>20:
      max_car=20

    #하이퍼파라미터
    genetic_config={
        'population_size':2,
        'n_gen': 1,
        'init_rate':0.3,
        'mutation_rate':0.5,
        'crossover_rate':0.3,
        'max_car_num':max_car
      }

    ga = GeneticAlgorithm(sub_order,d_mat,t_mat, **genetic_config)

    best_chromosome, best_score = ga.find_solution()
    final.append(ga.get_route_dictionary(best_chromosome[-1]))

    return final

final_Monday = solution(c_index_Monday, order_Monday_cluster)
final_Tuesday = solution(c_index_Tuesday, order_Tuesday_cluster)
final_Wednesday = solution(c_index_Wednesday, order_Wednesday_cluster)
final_Thursday = solution(c_index_Thursday, order_Thursday_cluster)
final_Friday = solution(c_index_Friday, order_Friday_cluster)
final_Saturday = solution(c_index_Saturday, order_Saturday_cluster)
'''

"\n#전체 탐색\ndef solution(c_index, order_cluster):\n  final = []\n  stop_list = []\n  for k,x in enumerate(c_index[1:]):\n    index = [i for i in range(c_index[k],c_index[k+1])]\n    print('/'*30)\n    print(f'processing {k+1}th cluster. complexity: {len(index)}')\n\n    sub_order = order_cluster.loc[index]\n    sub_order.sort_values('Stop_ID', inplace=True)\n    sub_order.reset_index(drop=True, inplace=True)\n\n    stop_list = [0]+list(sub_order['Stop_ID'])\n    d_mat,t_mat = get_dist_mat(stop_list)\n\n    #maximum vehicles\n    total_weight = sub_order['weight'].sum()\n    total_plt = sub_order['plt'].sum()\n    max_car_w = total_weight/1000\n    max_car_p = total_plt/2.8\n    max_car = int(max(max_car_w, max_car_p)+1)\n    if max_car>20:\n      max_car=20\n\n    #하이퍼파라미터\n    genetic_config={\n        'population_size':2,\n        'n_gen': 1,\n        'init_rate':0.3,\n        'mutation_rate':0.5,\n        'crossover_rate':0.3,\n        'max_car_num':max_car\n      }\n\n    ga = Genet

In [32]:
final_Monday=[]

#전체 탐색
for k,x in enumerate(c_index_Monday[1:]):
  index = [i for i in range(c_index_Monday[k],c_index_Monday[k+1])]
  print('/'*30)
  print(f'processing {k+1}th cluster. complexity: {len(index)}')

  sub_order = order_Monday_cluster.loc[index]
  sub_order.sort_values('Stop_ID', inplace=True)
  sub_order.reset_index(drop=True, inplace=True)

  stop_list = [0]+list(sub_order['Stop_ID'])
  d_mat,t_mat = get_dist_mat(stop_list)

  #maximum vehicles
  total_weight = sub_order['weight'].sum()
  total_plt = sub_order['plt'].sum()
  max_car_w = total_weight/1000
  max_car_p = total_plt/2.8
  max_car = int(max(max_car_w, max_car_p)+1)
  if max_car>20:
    max_car=20

  #하이퍼파라미터
  genetic_config={
      'population_size': 2,
      'n_gen': 1,
      'init_rate':0.3,
      'mutation_rate':0.5,
      'crossover_rate':0.3,
      'max_car_num':max_car
    }

  ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

  best_chromosome, best_score = ga.find_solution()
  final_Monday.append(ga.get_route_dictionary(best_chromosome[-1]))

//////////////////////////////
processing 1th cluster. complexity: 55
Best score in generation 1 : 0.1981032067586811
//////////////////////////////
processing 2th cluster. complexity: 38
Best score in generation 1 : 0.2999968671877364
//////////////////////////////
processing 3th cluster. complexity: 17
Best score in generation 1 : 0.4885630896781602
//////////////////////////////
processing 4th cluster. complexity: 12
Best score in generation 1 : 1.502453529530342
//////////////////////////////
processing 5th cluster. complexity: 51
Best score in generation 1 : 0.22670987419957198
//////////////////////////////
processing 6th cluster. complexity: 13
Best score in generation 1 : 0.9656449243650023
//////////////////////////////
processing 7th cluster. complexity: 29
Best score in generation 1 : 0.38937407864555346
//////////////////////////////
processing 8th cluster. complexity: 32
Best score in generation 1 : 0.41742558616224573
//////////////////////////////
processing 9th cluster.

In [33]:
final_Tuesday=[]

#전체 탐색
for k,x in enumerate(c_index_Tuesday[1:]):
  index = [i for i in range(c_index_Tuesday[k],c_index_Tuesday[k+1])]
  print('/'*30)
  print(f'processing {k+1}th cluster. complexity: {len(index)}')

  sub_order = order_Tuesday_cluster.loc[index]
  sub_order.sort_values('Stop_ID', inplace=True)
  sub_order.reset_index(drop=True, inplace=True)

  stop_list = [0]+list(sub_order['Stop_ID'])
  d_mat,t_mat = get_dist_mat(stop_list)

  #maximum vehicles
  total_weight = sub_order['weight'].sum()
  total_plt = sub_order['plt'].sum()
  max_car_w = total_weight/1000
  max_car_p = total_plt/2.8
  max_car = int(max(max_car_w, max_car_p)+1)
  if max_car>20:
    max_car=20

  #하이퍼파라미터
  genetic_config={
      'population_size': 2,
      'n_gen': 1,
      'init_rate':0.3,
      'mutation_rate':0.5,
      'crossover_rate':0.3,
      'max_car_num':max_car
    }

  ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

  best_chromosome, best_score = ga.find_solution()
  final_Tuesday.append(ga.get_route_dictionary(best_chromosome[-1]))

//////////////////////////////
processing 1th cluster. complexity: 66
Best score in generation 1 : 0.16205279950558463
//////////////////////////////
processing 2th cluster. complexity: 30
Best score in generation 1 : 0.4219155637057019
//////////////////////////////
processing 3th cluster. complexity: 44
Best score in generation 1 : 0.236176655528326
//////////////////////////////
processing 4th cluster. complexity: 7
Best score in generation 1 : 1.3869774988566685
//////////////////////////////
processing 5th cluster. complexity: 7
Best score in generation 1 : 1.0546146842119133
//////////////////////////////
processing 6th cluster. complexity: 16
Best score in generation 1 : 0.6933556172900947
//////////////////////////////
processing 7th cluster. complexity: 54
Best score in generation 1 : 0.21231606183844262
//////////////////////////////
processing 8th cluster. complexity: 31
Best score in generation 1 : 0.3139560185088328
//////////////////////////////
processing 9th cluster. co

In [34]:
final_Wednesday=[]

#전체 탐색
for k,x in enumerate(c_index_Wednesday[1:]):
  index = [i for i in range(c_index_Wednesday[k],c_index_Wednesday[k+1])]
  print('/'*30)
  print(f'processing {k+1}th cluster. complexity: {len(index)}')

  sub_order = order_Wednesday_cluster.loc[index]
  sub_order.sort_values('Stop_ID', inplace=True)
  sub_order.reset_index(drop=True, inplace=True)

  stop_list = [0]+list(sub_order['Stop_ID'])
  d_mat,t_mat = get_dist_mat(stop_list)

  #maximum vehicles
  total_weight = sub_order['weight'].sum()
  total_plt = sub_order['plt'].sum()
  max_car_w = total_weight/1000
  max_car_p = total_plt/2.8
  max_car = int(max(max_car_w, max_car_p)+1)
  if max_car>20:
    max_car=20

  #하이퍼파라미터
  genetic_config={
      'population_size': 2,
      'n_gen': 1,
      'init_rate':0.3,
      'mutation_rate':0.5,
      'crossover_rate':0.3,
      'max_car_num':max_car
    }

  ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

  best_chromosome, best_score = ga.find_solution()
  final_Wednesday.append(ga.get_route_dictionary(best_chromosome[-1]))

//////////////////////////////
processing 1th cluster. complexity: 38
Best score in generation 1 : 0.2844381864380901
//////////////////////////////
processing 2th cluster. complexity: 63
Best score in generation 1 : 0.18321316604779278
//////////////////////////////
processing 3th cluster. complexity: 17
Best score in generation 1 : 0.8778657078373945
//////////////////////////////
processing 4th cluster. complexity: 7
Best score in generation 1 : 1.0786527760611797
//////////////////////////////
processing 5th cluster. complexity: 51
Best score in generation 1 : 0.24354315306701418
//////////////////////////////
processing 6th cluster. complexity: 20
Best score in generation 1 : 0.4829315190382825
//////////////////////////////
processing 7th cluster. complexity: 14
Best score in generation 1 : 0.8274672299057277
//////////////////////////////
processing 8th cluster. complexity: 11
Best score in generation 1 : 1.3081201565469303
//////////////////////////////
processing 9th cluster. 

In [ ]:
final_Thursday=[]

#전체 탐색
for k,x in enumerate(c_index_Thursday[1:]):
  index = [i for i in range(c_index_Thursday[k],c_index_Thursday[k+1])]
  print('/'*30)
  print(f'processing {k+1}th cluster. complexity: {len(index)}')

  sub_order = order_Thursday_cluster.loc[index]
  sub_order.sort_values('Stop_ID', inplace=True)
  sub_order.reset_index(drop=True, inplace=True)

  stop_list = [0]+list(sub_order['Stop_ID'])
  d_mat,t_mat = get_dist_mat(stop_list)

  #maximum vehicles
  total_weight = sub_order['weight'].sum()
  total_plt = sub_order['plt'].sum()
  max_car_w = total_weight/1000
  max_car_p = total_plt/2.8
  max_car = int(max(max_car_w, max_car_p)+1)
  if max_car>20:
    max_car=20

  #하이퍼파라미터
  genetic_config={
      'population_size': 2,
      'n_gen': 1,
      'init_rate':0.3,
      'mutation_rate':0.5,
      'crossover_rate':0.3,
      'max_car_num':max_car
    }

  ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

  best_chromosome, best_score = ga.find_solution()
  final_Thursday.append(ga.get_route_dictionary(best_chromosome[-1]))

//////////////////////////////
processing 1th cluster. complexity: 35
Best score in generation 1 : 0.3631593188048618
//////////////////////////////
processing 2th cluster. complexity: 14
Best score in generation 1 : 0.7192826924694602
//////////////////////////////
processing 3th cluster. complexity: 28
Best score in generation 1 : 0.45354648309008994
//////////////////////////////
processing 4th cluster. complexity: 41
Best score in generation 1 : 0.3469682424227097
//////////////////////////////
processing 5th cluster. complexity: 44
Best score in generation 1 : 0.2483134846779989
//////////////////////////////
processing 6th cluster. complexity: 12
Best score in generation 1 : 0.6715106337001013
//////////////////////////////
processing 7th cluster. complexity: 23
Best score in generation 1 : 0.38922782039470544
//////////////////////////////
processing 8th cluster. complexity: 7
Best score in generation 1 : 2.1161375015814694
//////////////////////////////
processing 9th cluster. 

In [ ]:
final_Friday=[]

#전체 탐색
for k,x in enumerate(c_index_Friday[1:]):
  index = [i for i in range(c_index_Friday[k],c_index_Friday[k+1])]
  print('/'*30)
  print(f'processing {k+1}th cluster. complexity: {len(index)}')

  sub_order = order_Friday_cluster.loc[index]
  sub_order.sort_values('Stop_ID', inplace=True)
  sub_order.reset_index(drop=True, inplace=True)

  stop_list = [0]+list(sub_order['Stop_ID'])
  d_mat,t_mat = get_dist_mat(stop_list)

  #maximum vehicles
  total_weight = sub_order['weight'].sum()
  total_plt = sub_order['plt'].sum()
  max_car_w = total_weight/1000
  max_car_p = total_plt/2.8
  max_car = int(max(max_car_w, max_car_p)+1)
  if max_car>20:
    max_car=20

  #하이퍼파라미터
  genetic_config={
      'population_size': 2,
      'n_gen': 1,
      'init_rate':0.3,
      'mutation_rate':0.5,
      'crossover_rate':0.3,
      'max_car_num':max_car
    }

  ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

  best_chromosome, best_score = ga.find_solution()
  final_Friday.append(ga.get_route_dictionary(best_chromosome[-1]))

In [ ]:
final_Saturday=[]

#전체 탐색
for k,x in enumerate(c_index_Saturday[1:]):
  index = [i for i in range(c_index_Saturday[k],c_index_Saturday[k+1])]
  print('/'*30)
  print(f'processing {k+1}th cluster. complexity: {len(index)}')

  sub_order = order_Friday_cluster.loc[index]
  sub_order.sort_values('Stop_ID', inplace=True)
  sub_order.reset_index(drop=True, inplace=True)

  stop_list = [0]+list(sub_order['Stop_ID'])
  d_mat,t_mat = get_dist_mat(stop_list)

  #maximum vehicles
  total_weight = sub_order['weight'].sum()
  total_plt = sub_order['plt'].sum()
  max_car_w = total_weight/1000
  max_car_p = total_plt/2.8
  max_car = int(max(max_car_w, max_car_p)+1)
  if max_car>20:
    max_car=20

  #하이퍼파라미터
  genetic_config={
      'population_size': 2,
      'n_gen': 1,
      'init_rate':0.3,
      'mutation_rate':0.5,
      'crossover_rate':0.3,
      'max_car_num':max_car
    }

  ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

  best_chromosome, best_score = ga.find_solution()
  final_Saturday.append(ga.get_route_dictionary(best_chromosome[-1]))

In [ ]:
def transform_keys(final):
    new_final = []

    for d in final:
        new_dict = {}
        key_groups = {}

        # Group keys by their 100s range
        for key in d.keys():
            base_key = (key // 100) * 100
            if base_key not in key_groups:
                key_groups[base_key] = []
            key_groups[base_key].append(key)

        # Transform keys within each group
        for base_key, keys in key_groups.items():
            for i, old_key in enumerate(sorted(keys)):
                new_key = base_key + i
                new_dict[new_key] = d[old_key]

        new_final.append(new_dict)

    return new_final


new_final_Monday = transform_keys(final_Monday)
new_final_Tuesday = transform_keys(final_Tuesday)
new_final_Wednesday = transform_keys(final_Wednesday)
new_final_Thursday = transform_keys(final_Thursday)
new_final_Friday = transform_keys(final_Friday)
new_final_Saturday = transform_keys(final_Saturday)



In [ ]:
order_stop_id_Monday = []
for i in range(len(final_Monday)):
  stop_id_list_Monday = order_Monday_cluster_groupbyid[i]['Stop_ID'].tolist()
  order_stop_id_Monday.append(stop_id_list_Monday)

order_stop_id_Tuesday = []
for i in range(len(final_Tuesday)):
    stop_id_list_Tuesday = order_Tuesday_cluster_groupbyid[i]['Stop_ID'].tolist()
    order_stop_id_Tuesday.append(stop_id_list_Tuesday)

order_stop_id_Wednesday = []
for i in range(len(final_Wednesday)):
    stop_id_list_Wednesday = order_Wednesday_cluster_groupbyid[i]['Stop_ID'].tolist()
    order_stop_id_Wednesday.append(stop_id_list_Wednesday)

order_stop_id_Thursday = []
for i in range(len(final_Thursday)):
    stop_id_list_Thursday = order_Thursday_cluster_groupbyid[i]['Stop_ID'].tolist()
    order_stop_id_Thursday.append(stop_id_list_Thursday)

order_stop_id_Friday = []
for i in range(len(final_Friday)):
    stop_id_list_Friday = order_Friday_cluster_groupbyid[i]['Stop_ID'].tolist()
    order_stop_id_Friday.append(stop_id_list_Friday)

order_stop_id_Saturday = []
for i in range(len(final_Saturday)):
    stop_id_list_Saturday = order_Saturday_cluster_groupbyid[i]['Stop_ID'].tolist()
    order_stop_id_Saturday.append(stop_id_list_Saturday)


In [ ]:
#Transform the values in final using order_stop_id
fin_final_Monday = new_final_Monday
fin_final_Tuesday = new_final_Tuesday
fin_final_Wednesday = new_final_Wednesday
fin_final_Thursday = new_final_Thursday
fin_final_Friday = new_final_Friday
fin_final_Saturday = new_final_Saturday


# 특정 키의 인덱스를 찾는 함수
def key_index(d, target_key):
    keys_list = list(d.keys())
    try:
        return keys_list.index(target_key)
    except ValueError:
        return -1  # 키가 딕셔너리에 없으면 -1 반환


for i in range(len(fin_final_Monday)):
    for key in fin_final_Monday[i].keys():
      fin_final_Monday[i][key] = order_stop_id_Monday[i][key_index(fin_final_Monday[i], key)]

for i in range(len(fin_final_Tuesday)):
    for key in fin_final_Tuesday[i].keys():
        fin_final_Tuesday[i][key] = order_stop_id_Tuesday[i][key_index(fin_final_Tuesday[i], key)]

for i in range(len(fin_final_Wednesday)):
    for key in fin_final_Wednesday[i].keys():
        fin_final_Wednesday[i][key] = order_stop_id_Wednesday[i][key_index(fin_final_Wednesday[i], key)]

for i in range(len(fin_final_Thursday)):
    for key in fin_final_Thursday[i].keys():
        fin_final_Thursday[i][key] = order_stop_id_Thursday[i][key_index(fin_final_Thursday[i], key)]

for i in range(len(fin_final_Friday)):
    for key in fin_final_Friday[i].keys():
        fin_final_Friday[i][key] = order_stop_id_Friday[i][key_index(fin_final_Friday[i], key)]

for i in range(len(fin_final_Saturday)):
    for key in fin_final_Saturday[i].keys():
        fin_final_Saturday[i][key] = order_stop_id_Saturday[i][key_index(fin_final_Saturday[i], key)]



# Print the transformed new_final
for d in fin_final:
    print(d)




In [ ]:
m = folium.Map(location=[37.58, 127.0], tiles='openstreetmap', zoom_start=11)



def get_map(df, index_list):
  """
  index_list=[]
  for i in stop_list:
    index = order_Monday_adj[order_Monday_adj['Stop_ID']==i].index[0]
    index_list.append(index)
  """

  m = folium.Map(location=[37.58, 127.0], tiles='openstreetmap', zoom_start=11)

  for _, row in df.loc[index_list].iterrows():

      Circle(location = [row['Y'], row['X']],
            popup=(row['Stop_ID'], row['weight']),
            radius = row['weight']/3,
            ).add_to(m)
      """
      Marker(location = [row['Y'],row['X']],
            popup = row['Stop_ID'],
            icon=folium.Icon(color='blue',icon='star')
            ).add_to(m)
      """

  Marker(location = [center['Y'],center['X']],
    popup = 'center',
    icon=folium.Icon(color='red',icon='star')
    ).add_to(m)

  return m

In [ ]:
#testing k th cluster
k=4
index = [i for i in range(c_index[k],c_index[k+1])]

sub_order = order_Monday_cluster.loc[index]
sub_order.sort_values('Stop_ID', inplace=True)
sub_order.reset_index(drop=True, inplace=True)

stop_list = [0]+list(sub_order['Stop_ID'])
d_mat,t_mat = get_dist_mat(stop_list)

#maximum vehicles
total_weight = sub_order['weight'].sum()
total_plt = sub_order['plt'].sum()
max_car_w = total_weight/1000
max_car_p = total_plt/2.8
max_car = int(max(max_car_w, max_car_p)+1)

genetic_config={
  'population_size':50,
  'n_gen':30,
  'init_rate':0.3,
  'mutation_rate':0.5,
  'crossover_rate':0.3,
  'max_car_num':max_car
}

ga = GeneticAlgorithm(sub_order,d_mat,t_mat,**genetic_config)

In [ ]:
'''
final=[]
for _ in range(1):
  best_chromosome, best_score = ga.find_solution()
  final.append(ga.get_route_dictionary(best_chromosome[-1]))
'''


디버깅 테스트